# COVID-19 AUSTRIA STATISTICS

In [1]:
import urllib.request
url = 'https://raw.githubusercontent.com/covid19-eu-zh/covid19-eu-data/master/dataset/covid-19-at.csv'
filename = 'covid_data/covid-19-at.csv'
urllib.request.urlretrieve(url, filename)

('covid_data/covid-19-at.csv', <http.client.HTTPMessage at 0x28fb3f7c390>)

In [2]:
import csv
import datetime
from datetime import date

today = date.today()
today = today.strftime("%y-%m-%d")
print("Today's date:", today)

safe_date = date.today()-datetime.timedelta(2)
safe = safe_date.strftime("%y-%m-%d")
print("Safe date:", safe)

date_time_str = '2018-06-29T08:15:27'
date_time_obj = datetime.datetime.strptime(date_time_str, '%Y-%m-%dT%H:%M:%S')

with open('covid_data/covid-19-at.csv', 'r') as i, open('covid_data/covid-19-austria.csv', 'w', newline='') as o:
    r = csv.reader(i, delimiter=',')
    writer = csv.writer(o)
    writer.writerow(["country", "name", "cases", "recovered", "deaths", "tests", "hospitalized", "intensive_care", "datetime"])
    #for row in r:
    #    if row[8] == "2020-05-09T15:00:00": 
    #        writer.writerow(row)
    #        print(row)
    max_value = date_time_obj
    #most recent date
    for row in r:
        if today in row[8]:
            date_time = row[8]
            date_time_object = datetime.datetime.strptime(date_time, '%Y-%m-%dT%H:%M:%S')
            if date_time_object > max_value: 
                max_value = date_time_object
    print("Most recent date: ", max_value)
    
                
                
                
with open('covid_data/covid-19-at.csv', 'r') as i, open('covid_data/covid-19-austria.csv', 'w', newline='') as o:
    r = csv.reader(i, delimiter=',')
    writer = csv.writer(o)
    writer.writerow(["country", "name", "cases", "recovered", "deaths", "tests", "hospitalized", "intensive_care", "datetime"])
                
    #create csv with most recent data           
    for row in r:
        if today in row[8]:
            date_time = row[8]
            date_time_object = datetime.datetime.strptime(date_time, '%Y-%m-%dT%H:%M:%S')
            if date_time_object == max_value: 
                writer.writerow(row)
                print(row)    
        

    

Today's date: 20-05-31
Safe date: 20-05-29
Most recent date:  2020-05-31 15:30:00
['AT', 'Burgenland', '342', '325', '11', '12953', '2', '0', '2020-05-31T15:30:00']
['AT', 'KÃ¤rnten', '414', '398', '13', '17104', '0', '0', '2020-05-31T15:30:00']
['AT', 'Vorarlberg', '898', '866', '19', '21763', '0', '0', '2020-05-31T15:30:00']
['AT', 'Salzburg', '1220', '1196', '38', '33275', '0', '0', '2020-05-31T15:30:00']
['AT', 'Steiermark', '1836', '1645', '147', '50817', '6', '0', '2020-05-31T15:30:00']
['AT', 'OberÃ¶sterreich', '2296', '2215', '60', '62819', '7', '0', '2020-05-31T15:30:00']
['AT', 'NiederÃ¶sterreich', '2866', '2673', '100', '69621', '22', '10', '2020-05-31T15:30:00']
['AT', 'Wien', '3229', '2855', '172', '106863', '53', '14', '2020-05-31T15:30:00']
['AT', 'Tirol', '3537', '3420', '108', '73319', '7', '3', '2020-05-31T15:30:00']
['AT', '', '16638', '15593', '668', '448534', '97', '27', '2020-05-31T15:30:00']


In [4]:
import matplotlib.pyplot as plt
import pandas as pd

# Import Data
df_raw = pd.read_csv("covid_data/covid-19-austria.csv")

# Prepare Data

name = df_raw['name'].tolist()[0:9]
intensive_care = df_raw['intensive_care'].tolist()[0:9]
cases = df_raw['cases'].tolist()[0:9]
deaths = df_raw['deaths'].tolist()[0:9]
recovered = df_raw['recovered'].tolist()[0:9]
hospitalized = df_raw['hospitalized'].tolist()[0:9]
tests = df_raw['tests'].tolist()[0:9]